## Unicode and Python

#### Mostly Wrong Historical Background

In the begining only America existed (kind of, in technology universe). And computer memory was expensive. Terms like megabyte, gigabyte were unheard of (little exaggeration is allowed here).

At that time (during 1960s), telegraphy was a prevalent way for long distance communication and it used 7 bits telgraph code. You know, computers don't talk in decimals like we do. They talk in binary. So when you say, 'I have 10 apples', computer would hear that you have 1010 apples and it would need 4 bits to save this 1010 binary number. 

Each bit can either hold '0' or '1'. So a 7 bit storage system can store values between '0000000' and '1111111'. Remember these are binary numbers, not your usual decimal numbers. In our language, computer can store numbers from 0 to 127 (128 numbers) in 7 bit storage system. 128 isn't a big number per se but back then it was sufficient to store the characters Americans care about (52 for A-Z and a-z, 10 for digits, some for symbols like '&' and other for non printable control characters). This way they consumed all the possible values 7 bit storage system could consume. Later on, ASCII standard was formalized based on this code and it became widely accepted standard. So far so good. 

Later on, 8-bit computers became widely available. Going by the explanation above, you can see 8 bit of data can be translaed into 256 values (0 - 255). Of these, first 128 were already used by ASCII standard. Different organizations/countries used rest values (128 - 255) to develop their own standard. For example, Russia used these values to assign letters from Russian alphabets. Ditto was true, say, for France. Or IBM vs other organization. Also, with the passage of time, more and more characters were needed to be incorporated in computer system. Afterall, many other countries started using computers.  

Can you see the problem here? One problem is portability. A document written in one standard is useless in another place because at other place, another standard is in practice. There were other problems as well (like efficient use of memory). Of course, actual histoy is much more complicated then presented here but you get the idea. 

Let us give you some actual taste. To deal with non-ASCII characters, Windows has the concept of `code page`. It has a collection of code pages. Each code page deals with some language specific set of characters. 

Below, I ran a Windows command `chcp` and its output was `Active code page: 1252`. However, the command was run *in Jupyter Notebook environment*. This code page can deal with West European Latin characters. 

In [3]:
!chcp 

Active code page: 437


If you run `chcp` command in actual Command Prompt as shown below in the snapshot, you'll see that the ouptut was `437`. If you try to print the same character `Ì` there using Python command, it will print non-accented `I` which is actually wrong (actually if you set chcp to 1252, things get even worse. Now it will print some garbage rather than non-accented `I`)

<img src = 'images/chr204.png' width = '500' height = '300'/>

In short, this is a total mess. 

#### Enter Unicode

The ASCII standard was about only 128 characters. The Unicode 11.0 standard deals with 137,439 characters. 

In Unicode, every character is assigned a code point. This code point is basically a hexadecimal integer. For example, 
the letter 'w' is assigned the `U+0077` code point. The part `U+` stands for Unicode. The important part of this code point is the part `'0077'`. This is a hexadecimal number rather than a decimal number. In programming terminology, hexadecimal number is denoted like 0x to distinguish it from decimal number. So here `0077` is actually `0x77`. The decimal value of `0x77` is `119`. So we say character 'w' has the code point `U+0077` with hexadecimal value `0x77` (`119` decimal.) 

In [4]:
print('\u0077') #you can print 'w' like this
print(hex(119)) # converting decimal to hexadecimal
a = int('0x0077', 16) #converting hexadecimal string to hexadecimal number 
print(a)
print(ord('\u0077')) #ord function returns the decimal unicode code point for single character. 
print(ord('w'))      #same as above. Computer doesn't differentiate between '\u0077' and 'w'.
print(chr(119))   # given code point, chr function returns the unicode string of one character
print(chr(0x0077)) # same as above

w
0x77
119
119
119
w
w


In [3]:
"\N{GREEK CAPITAL LETTER DELTA}" #you can use unicode name of character as well

'Δ'

This was some high level description of Unicode. Let's go a bit low level. So far we talked about only 2 standards: ASCII and its extension Unicode (before I forget, let me tell you that ASCII text is always a Unicode text as well, though opposite isn't true.) There are dozens of such standards. ASCII and Unicode are just 2 of them.

You know, computer stores everything in bytes containing 0s and 1s. Image files, text files, python program files (.py), PDF files: everything is stored as a one long sequence of 1s and 0s. So how  does computer recognize various file types if it is all 0s and 1s down there? Well, computer recognizes these files with their file extensions and opens them with relevant software/program because only those software/program can *decode those 0s and 1s in meaningful manner*

The same is true about characters. Ultimately characters are converted (encoded) into those 1s and 0s and which then are saved in bytes and you need correct decoding program to make use of those bytes. Character to byte conversion is done by a set of rules called encoding. The `UTF-8` is the most prevalent encoding system. Under `UTF-8`, All characters are encoded in 1-4 bytes, depending on their code point value. 

See following table - 

Code point range(dec)|code point range(hex)| byte size
---------------|---------------|---------------------
0 - 127|0X0 - 0X7F|1 byte
128 - 2047|0X0080 - 0X7FF|2 byte
2048 - 65535|0X0800 - 0XFFFF|3 byte
65536 - 1114111|0X10000 - 0X10FFFF|4 byte

The x characters are replaced by the bits of the code point. If the number of significant bits is no more than seven, the first line applies; if no more than 11 bits, the second line applies, and so on. 

<img src = 'images\utf.png' width = '600' height ='400'/>
In above table you'll notice that for each multibytes sequence, all bytes other than first bytes always start with first 2 bits set to `10`. This is a way to let us know that we are in the middle of a multibyte sequence. If you see `11` anywhere in multibytes sequence, you'll know you are at the start of multibytes sequence. 

The leading bytes and the continuation bytes do not share values (continuation bytes start with `10` while single bytes start with 0 and longer lead bytes start with 11). This means a search will not accidentally find the sequence for one character starting in the middle of another character. It also means the start of a character can be found from a random position by backing up at most 3 bytes to find the leading byte. An incorrect character will not be decoded if a stream starts mid-sequence, and a shorter sequence will never appear inside a longer one.


Now let us talk about some character with decimal code point greater than 127, say 204. Obviously it will need 2 bytes to store. Let us explore this character.

In [5]:
print(chr(204)) #this character represents accented capital I. 
print(hex(204))

Ì
0xcc


In [6]:
chr(0xcc) #hexadecimal convert of 204. 

'Ì'

In [7]:
print('\u00cc') #printing unicode point

Ì


Since string is what you deal with and bytes are what computers store, we need some way to convert one into another. The rule-set which is followed for this back-and-forth conversion is UTF-8 (among many other encodings). The conversion of string into bytes is called 'encoding' while conversion from bytes to string is called decoding. 

Python provides 2 methods to accomplish these conversions: `encode()` and `decode()`.

    string.encode() = bytes
    bytes.decode() = string
    
 Actually these methods will work with other encodings as well. You just have to pass the encoding as parameter in the method. Let us play with the same character with code point 204 (decimal).  

In [8]:
'Ì'.encode() #default encoding is utf-8

b'\xc3\x8c'

In [9]:
'\u00cc'.encode() #code point as string

b'\xc3\x8c'

In [10]:
b'\xc3\x8c'.decode() # bytes to string conversion. again, default encoding is utf-8

'Ì'

In [11]:
'\u00cc'.encode('ascii') #will throw error

UnicodeEncodeError: 'ascii' codec can't encode character '\xcc' in position 0: ordinal not in range(128)

As was mentioned earlier, for code point 0 to 127, there is no difference between `ascii` text and `unicode` text. But here, we are dealing with the character '`\u00cc`' which has the code point 204 and this value is greater than 127, which means this character can't be encoded with `ascii` encoding.

Also, this character will need 2 bytes in memory and encoded bytes `b'\xc3\x8c'` actually tell the same thing. These bytes have 2 hexadecimal digits in: `0xc3` and `0x8c`. 

Earlier we saw decimal 204 equals to hex `0xcc`. Converting this into binary representation - 

In [12]:
bin(204), bin(0xcc) #both are same

('0b11001100', '0b11001100')

The binary representation of 204 (or `oxcc`) is 8 bits long. However, as per the chart shown above, we actually need 11 bits to store the bits (5 in first byte and 6 in second byte), so we add 3 leading 0 to our binary number `11001100`. This means now the binary representation of decimal 204 is `00011001100`. Of this sequence, we pick the leading 5 bits, that is, `00011` and put it after `110` in first byte. 

Now, the second byte has space for 6 bits (after leading bits of `10`). So we fill these space with remaining bits, that is, `001100`. So we have something like this -

`110-00011` - first byte 

`10-001100` - second byte

Now let us convert these bytes into hexadecimal number -

In [13]:
hex(0b11000011), hex(0b10001100)

('0xc3', '0x8c')

These are the same hex number whichs we saw when we encoded the character `Ì` - 

In [14]:
'Ì'.encode()

b'\xc3\x8c'